# Manutenzione preventiva - DoS Attacks

In [1]:
import pandas as pd
import json as json
import ipaddress as ip 
import numpy as np
import warnings
import functions_notebook_new as fn
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC


warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
### costanti
# clean captures
clean_capture = 'captures\\captures1_v2\\clean\\eth2dump-clean-0,5h_1.json'

# ping flooding captures 1 min
pf_capture_1 = 'captures\\captures1_v2\\pingFloodDDoS\\eth2dump-pingFloodDDoS1m-0,5h_1.json'
pf_capture_2 = 'captures\\captures1_v2\\pingFloodDDoS\\eth2dump-pingFloodDDoS5m-0,5h_1.json'
pf_capture_3 = 'captures\\captures1_v2\\pingFloodDDoS\\eth2dump-pingFloodDDoS15m-0,5h_1.json'

levels_of_interest = ['frame', 'eth', 'tcp', 'arp', 'udp', 'modbus', 'icmp', 'ip']

### variabili

In [3]:
### estrazione dati
# packets clean capture
packets_cc = fn.load_packets_from_file(clean_capture)

# packets attack captures (es. packets ping flooding attack 1m per cattura di 0.5 = pkt_pf_1m_05h)
pkt_pf_1m_05h = fn.load_packets_from_file(pf_capture_1)
pkt_pf_5m_05h = fn.load_packets_from_file(pf_capture_2)
pkt_pf_15m_05h = fn.load_packets_from_file(pf_capture_3)

In [4]:
### features
# clean features
features_cc = fn.extract_features_from_packets(packets_cc, levels_of_interest)

# ping flooding attack features
features_pf_1m_05h = fn.extract_features_from_packets(pkt_pf_1m_05h, levels_of_interest)
features_pf_5m_05h = fn.extract_features_from_packets(pkt_pf_5m_05h, levels_of_interest)
features_pf_15m_05h =fn.extract_features_from_packets(pkt_pf_15m_05h, levels_of_interest)

In [5]:
### Dataframes
# df clean capture
df_clean = pd.DataFrame(features_cc)

# df ping flooding attacks captures
df_pf_1m_05h = pd.DataFrame(features_pf_1m_05h)
df_pf_5m_05h = pd.DataFrame(features_pf_5m_05h)
df_pf_15m_05h = pd.DataFrame(features_pf_15m_05h)


pd_concatenation = pd.concat([df_clean, df_pf_1m_05h, df_pf_5m_05h, df_pf_15m_05h])

In [6]:
pd_concatenation.isna().sum().to_excel('check_nulls_number.xlsx')

PermissionError: [Errno 13] Permission denied: 'check_nulls_number.xlsx'

In [14]:
df_colonne_senza_nan = pd_concatenation.loc[:, pd_concatenation.isna().sum() == 0]

In [15]:
len(df_colonne_senza_nan.select_dtypes(include=['object']).columns)

60

In [16]:
df_colonne_senza_nan.select_dtypes(include=['object']).iloc[:10,50:60]

,eth.eth.src_tree.eth.addr.oui_resolved_raw[0],eth.eth.src_tree.eth.addr.oui_resolved,eth.eth.src_tree.eth.src.lg_raw[0],eth.eth.src_tree.eth.src.lg,eth.eth.src_tree.eth.lg_raw[0],eth.eth.src_tree.eth.lg,eth.eth.src_tree.eth.src.ig_raw[0],eth.eth.src_tree.eth.src.ig,eth.eth.src_tree.eth.ig_raw[0],eth.eth.src_tree.eth.ig
0,000c299d9e9e,"VMware, Inc.",0,0,0,0,0,0,0,0
1,d07e288e40b3,Hewlett Packard,0,0,0,0,0,0,0,0
2,000c299d9e9e,"VMware, Inc.",0,0,0,0,0,0,0,0
3,0080f409513b,Telemecanique Electrique,0,0,0,0,0,0,0,0
4,000c299d9e9e,"VMware, Inc.",0,0,0,0,0,0,0,0
5,000c299d9e9e,"VMware, Inc.",0,0,0,0,0,0,0,0
6,0080f409513b,Telemecanique Electrique,0,0,0,0,0,0,0,0
7,485b39644079,ASUSTek COMPUTER INC.,0,0,0,0,0,0,0,0
8,0080f409513b,Telemecanique Electrique,0,0,0,0,0,0,0,0
9,0080f409513b,Telemecanique Electrique,0,0,0,0,0,0,0,0


### Feature tipo intero
frame.frame.encap_type
frame.frame.number
frame.frame.len
eth.eth.dst_tree.eth.dst.oui	
frame.frame.cap_len	frame.frame.marked	frame.frame.ignored
eth.eth.dst_tree.eth.dst.lg_raw[0]	
eth.eth.dst_tree.eth.addr.oui
eth.eth.dst_tree.eth.dst.lg
eth.eth.dst_tree.eth.lg_raw[0]	eth.eth.dst_tree.eth.lg	eth.eth.dst_tree.eth.dst.ig_raw[0]	eth.eth.dst_tree.eth.dst.ig	eth.eth.dst_tree.eth.ig_raw[0]	eth.eth.dst_tree.eth.ig
eth.eth.src
eth.eth.src_tree.eth.src.oui	
eth.eth.src_tree.eth.addr.oui
eth.eth.src_tree.eth.src.lg_raw[0]	eth.eth.src_tree.eth.src.lg	eth.eth.src_tree.eth.lg_raw[0]	eth.eth.src_tree.eth.lg	eth.eth.src_tree.eth.src.ig_raw[0]	eth.eth.src_tree.eth.src.ig	eth.eth.src_tree.eth.ig_raw[0]	eth.eth.src_tree.eth.ig
#### ESADECIMALE
eth.eth.dst_raw[0]	 eth.eth.dst_tree.eth.dst_resolved_raw[0] eth.eth.dst_tree.eth.dst.oui_raw[0] eth.eth.dst_tree.eth.addr_raw[0] eth.eth.dst_tree.eth.addr.oui_raw[0]
eth.eth.src_raw[0]
eth.eth.src_tree.eth.src_resolved_raw[0]	
eth.eth.src_tree.eth.src.oui_raw[0]
eth.eth.src_tree.eth.src.oui_resolved_raw[0]
eth.eth.src_tree.eth.addr_raw[0]
eth.eth.src_tree.eth.addr
eth.eth.src_tree.eth.addr_resolved_raw[0]
eth.eth.src_tree.eth.addr.oui_raw[0]
eth.eth.src_tree.eth.addr.oui_resolved_raw[0]				
#### MAC ADDRESS	
eth.eth.dst mac address eth.eth.dst_tree.eth.addr eth.eth.dst_tree.eth.addr_resolved_raw[0] 	 (da rimuovere : e conversione in esadecimale)

### Feature tipo float
frame.frame.time_epoch
frame.frame.offset_shift
frame.frame.time_delta
frame.frame.time_delta_displayed
frame.frame.time_relative
### Feature tipo string
frame.frame.protocols	da ricavare i protocolli contenuti all'interno del frame
#### DA SCARTARE
'frame.frame.coloring_rule.name'
'frame.frame.coloring_rule.string'
'eth.eth.dst_tree.eth.dst_resolved'
'eth.eth.dst_tree.eth.addr_resolved'
'eth.eth.src_tree.eth.src_resolved'
'eth.eth.src_tree.eth.src.oui_resolved'	
'eth.eth.src_tree.eth.addr_resolved'
'eth.eth.src_tree.eth.addr.oui_resolved'

In [17]:
pd_nona_bck = df_colonne_senza_nan.copy()
df_colonne_senza_nan.drop(columns=['frame.frame.coloring_rule.name','frame.frame.coloring_rule.string', 'eth.eth.dst_tree.eth.dst_resolved', 'eth.eth.dst_tree.eth.addr_resolved', 'eth.eth.src_tree.eth.src_resolved', 'eth.eth.src_tree.eth.src.oui_resolved', 'eth.eth.src_tree.eth.addr_resolved', 'eth.eth.src_tree.eth.addr.oui_resolved'], inplace=True)

C:\Users\olivi\AppData\Local\Temp\ipykernel_16356\1994833931.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_colonne_senza_nan.drop(columns=['frame.frame.coloring_rule.name','frame.frame.coloring_rule.string', 'eth.eth.dst_tree.eth.dst_resolved', 'eth.eth.dst_tree.eth.addr_resolved', 'eth.eth.src_tree.eth.src_resolved', 'eth.eth.src_tree.eth.src.oui_resolved', 'eth.eth.src_tree.eth.addr_resolved', 'eth.eth.src_tree.eth.addr.oui_resolved'], inplace=True)


In [19]:
len(df_colonne_senza_nan)

340626

In [20]:
df_nonan_converted = fn.df_convert(df_colonne_senza_nan)

AttributeError: module 'functions_notebook_new' has no attribute 'df_convert'